In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [37]:
df = pd.read_csv("/content/deliveries.csv")

In [38]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1,2023/24,05-10-2023,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023/24,05-10-2023,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2023/24,05-10-2023,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2023/24,05-10-2023,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,JM Bairstow,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2023/24,05-10-2023,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
import pandas as pd

grouped_data = df.groupby(['match_id', 'batting_team', 'striker'])

total_runs_by_batsman = grouped_data['runs_off_bat'].sum().reset_index(name='total_runs_by_batsman')

bowlers_faced = grouped_data['bowler'].unique().reset_index(name='bowlers_faced')

bowlers_faced_expanded = pd.concat([bowlers_faced.drop('bowlers_faced', axis=1),
                                    bowlers_faced['bowlers_faced'].apply(lambda x: pd.Series(x, dtype=object))], axis=1)
column_names = {i: f'bowler_{i}' for i in range(bowlers_faced_expanded.shape[1] - 3)}  # Assuming 3 existing columns
bowlers_faced_expanded.rename(columns=column_names, inplace=True)

match_details = df.groupby(['match_id', 'batting_team']).agg({
    'venue': 'first',
    'bowling_team': 'first'
}).reset_index()

result_df = pd.merge(total_runs_by_batsman, match_details, on=['match_id', 'batting_team'])

result_df = pd.merge(result_df, bowlers_faced_expanded, on=['match_id', 'batting_team', 'striker'])

result_df

,match_id,batting_team,striker,total_runs_by_batsman,venue,bowling_team,bowler_0,bowler_1,bowler_2,bowler_3,bowler_4,bowler_5,bowler_6,bowler_7
0,1,England,AU Rashid,15,"Narendra Modi Stadium, Ahmedabad",New Zealand,MJ Henry,TA Boult,JDS Neesham,NaN,NaN,NaN,NaN,NaN
1,1,England,CR Woakes,11,"Narendra Modi Stadium, Ahmedabad",New Zealand,GD Phillips,R Ravindra,MJ Santner,NaN,NaN,NaN,NaN,NaN
2,1,England,DJ Malan,14,"Narendra Modi Stadium, Ahmedabad",New Zealand,TA Boult,MJ Henry,MJ Santner,NaN,NaN,NaN,NaN,NaN
3,1,England,HC Brook,25,"Narendra Modi Stadium, Ahmedabad",New Zealand,MJ Santner,TA Boult,JDS Neesham,R Ravindra,NaN,NaN,NaN,NaN
4,1,England,JC Buttler,43,"Narendra Modi Stadium, Ahmedabad",New Zealand,GD Phillips,R Ravindra,MJ Santner,TA Boult,JDS Neesham,MJ Henry,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,32,South Africa,DA Miller,53,"Maharashtra Cricket Association Stadium, Pune",New Zealand,MJ Santner,JDS Neesham,TG Southee,GD Phillips,TA Boult,NaN,NaN,NaN
581,32,South Africa,H Klaasen,15,"Maharashtra Cricket Association Stadium, Pune",New Zealand,TG Southee,TA Boult,JDS Neesham,NaN,NaN,NaN,NaN,NaN
582,32,South Africa,HE van der Dussen,133,"Maharashtra Cricket Association Stadium, Pune",New Zealand,TA Boult,TG Southee,MJ Santner,GD Phillips,R Ravindra,MJ Henry,JDS Neesham,NaN
583,32,South Africa,Q de Kock,114,"Maharashtra Cricket Association Stadium, Pune",New Zealand,TA Boult,MJ Henry,TG Southee,MJ Santner,GD Phillips,R Ravindra,JDS Neesham,NaN


In [40]:
result_df.head()

,match_id,batting_team,striker,total_runs_by_batsman,venue,bowling_team,bowler_0,bowler_1,bowler_2,bowler_3,bowler_4,bowler_5,bowler_6,bowler_7
0,1,England,AU Rashid,15,"Narendra Modi Stadium, Ahmedabad",New Zealand,MJ Henry,TA Boult,JDS Neesham,NaN,NaN,NaN,NaN,NaN
1,1,England,CR Woakes,11,"Narendra Modi Stadium, Ahmedabad",New Zealand,GD Phillips,R Ravindra,MJ Santner,NaN,NaN,NaN,NaN,NaN
2,1,England,DJ Malan,14,"Narendra Modi Stadium, Ahmedabad",New Zealand,TA Boult,MJ Henry,MJ Santner,NaN,NaN,NaN,NaN,NaN
3,1,England,HC Brook,25,"Narendra Modi Stadium, Ahmedabad",New Zealand,MJ Santner,TA Boult,JDS Neesham,R Ravindra,NaN,NaN,NaN,NaN
4,1,England,JC Buttler,43,"Narendra Modi Stadium, Ahmedabad",New Zealand,GD Phillips,R Ravindra,MJ Santner,TA Boult,JDS Neesham,MJ Henry,NaN,NaN


In [41]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

result_df['batting_team_encoded'] = label_encoder.fit_transform(result_df['batting_team'])

result_df['venue_encoded'] = label_encoder.fit_transform(result_df['venue'])

result_df['striker_encoded'] = label_encoder.fit_transform(result_df['striker'])

result_df['bowling_team_encoded'] = label_encoder.fit_transform(result_df['bowling_team'])

result_df[['bowler_0', 'bowler_1', 'bowler_2', 'bowler_3', 'bowler_4', 'bowler_5', 'bowler_6', 'bowler_7']] = result_df[['bowler_0', 'bowler_1', 'bowler_2', 'bowler_3', 'bowler_4', 'bowler_5', 'bowler_6', 'bowler_7']].fillna('Unknown')

# Apply label encoding
result_df['bowler_0_encoded'] = label_encoder.fit_transform(result_df['bowler_0'])
result_df['bowler_1_encoded'] = label_encoder.fit_transform(result_df['bowler_1'])
result_df['bowler_2_encoded'] = label_encoder.fit_transform(result_df['bowler_2'])
result_df['bowler_3_encoded'] = label_encoder.fit_transform(result_df['bowler_3'])
result_df['bowler_4_encoded'] = label_encoder.fit_transform(result_df['bowler_4'])
result_df['bowler_5_encoded'] = label_encoder.fit_transform(result_df['bowler_5'])
result_df['bowler_6_encoded'] = label_encoder.fit_transform(result_df['bowler_6'])
result_df['bowler_7_encoded'] = label_encoder.fit_transform(result_df['bowler_7'])

result_df = result_df.drop(['batting_team', 'venue', 'striker', 'bowling_team', 'bowler_0', 'bowler_1', 'bowler_2', 'bowler_3', 'bowler_4', 'bowler_5', 'bowler_6', 'bowler_7'], axis=1)

In [42]:
result_df.head()

,match_id,total_runs_by_batsman,batting_team_encoded,venue_encoded,striker_encoded,bowling_team_encoded,bowler_0_encoded,bowler_1_encoded,bowler_2_encoded,bowler_3_encoded,bowler_4_encoded,bowler_5_encoded,bowler_6_encoded,bowler_7_encoded
0,1,15,3,7,7,6,42,76,25,74,68,46,11,1
1,1,11,3,7,19,6,18,64,44,74,68,46,11,1
2,1,14,3,7,23,6,73,44,44,74,68,46,11,1
3,1,25,3,7,36,6,43,76,25,61,68,46,11,1
4,1,43,3,7,48,6,18,64,44,72,23,21,11,1


In [43]:
from sklearn.model_selection import train_test_split

X = result_df.drop('total_runs_by_batsman', axis=1)
y = result_df['total_runs_by_batsman']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

results_df = pd.DataFrame(columns=['Model', 'Mean Squared Error (MSE)', 'R-squared (R2)', 'Train RMSE', 'Test RMSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'])


In [46]:
# Linear Regression Model
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train_scaled, y_train)
y_pred_linear_reg = linear_reg_model.predict(X_test_scaled)
mse_linear_reg = mean_squared_error(y_test, y_pred_linear_reg)
r2_linear_reg = r2_score(y_test, y_pred_linear_reg)
train_rmse_linear_reg = mean_squared_error(y_train, linear_reg_model.predict(X_train_scaled), squared=False)
test_rmse_linear_reg = mean_squared_error(y_test, y_pred_linear_reg, squared=False)
train_mae_linear_reg = mean_absolute_error(y_train, linear_reg_model.predict(X_train_scaled))
test_mae_linear_reg = mean_absolute_error(y_test, y_pred_linear_reg)
train_r2_linear_reg = r2_score(y_train, linear_reg_model.predict(X_train_scaled))
test_r2_linear_reg = r2_score(y_test, y_pred_linear_reg)

results_df = results_df.append({
    'Model': 'Linear Regression',
    'Mean Squared Error (MSE)': mse_linear_reg,
    'R-squared (R2)': r2_linear_reg,
    'Train RMSE': train_rmse_linear_reg,
    'Test RMSE': test_rmse_linear_reg,
    'Train MAE': train_mae_linear_reg,
    'Test MAE': test_mae_linear_reg,
    'Train R2': train_r2_linear_reg,
    'Test R2': test_r2_linear_reg
}, ignore_index=True)


<ipython-input-46-d26b0bb95b4e>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [47]:
# SVM Regressor Model
svm_regressor_model = SVR()
svm_regressor_model.fit(X_train_scaled, y_train)
y_pred_svm_regressor = svm_regressor_model.predict(X_test_scaled)
mse_svm_regressor = mean_squared_error(y_test, y_pred_svm_regressor)
r2_svm_regressor = r2_score(y_test, y_pred_svm_regressor)
train_rmse_svm_regressor = mean_squared_error(y_train, svm_regressor_model.predict(X_train_scaled), squared=False)
test_rmse_svm_regressor = mean_squared_error(y_test, y_pred_svm_regressor, squared=False)
train_mae_svm_regressor = mean_absolute_error(y_train, svm_regressor_model.predict(X_train_scaled))
test_mae_svm_regressor = mean_absolute_error(y_test, y_pred_svm_regressor)
train_r2_svm_regressor = r2_score(y_train, svm_regressor_model.predict(X_train_scaled))
test_r2_svm_regressor = r2_score(y_test, y_pred_svm_regressor)

results_df = results_df.append({
    'Model': 'SVM Regressor',
    'Mean Squared Error (MSE)': mse_svm_regressor,
    'R-squared (R2)': r2_svm_regressor,
    'Train RMSE': train_rmse_svm_regressor,
    'Test RMSE': test_rmse_svm_regressor,
    'Train MAE': train_mae_svm_regressor,
    'Test MAE': test_mae_svm_regressor,
    'Train R2': train_r2_svm_regressor,
    'Test R2': test_r2_svm_regressor
}, ignore_index=True)

<ipython-input-47-984e9b3ea6d8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [48]:

# Decision Tree Regression Model
decision_tree_model = DecisionTreeRegressor(random_state=42)
decision_tree_model.fit(X_train_scaled, y_train)
y_pred_decision_tree = decision_tree_model.predict(X_test_scaled)
mse_decision_tree = mean_squared_error(y_test, y_pred_decision_tree)
r2_decision_tree = r2_score(y_test, y_pred_decision_tree)
train_rmse_decision_tree = mean_squared_error(y_train, decision_tree_model.predict(X_train_scaled), squared=False)
test_rmse_decision_tree = mean_squared_error(y_test, y_pred_decision_tree, squared=False)
train_mae_decision_tree = mean_absolute_error(y_train, decision_tree_model.predict(X_train_scaled))
test_mae_decision_tree = mean_absolute_error(y_test, y_pred_decision_tree)
train_r2_decision_tree = r2_score(y_train, decision_tree_model.predict(X_train_scaled))
test_r2_decision_tree = r2_score(y_test, y_pred_decision_tree)

results_df = results_df.append({
    'Model': 'Decision Tree Regression',
    'Mean Squared Error (MSE)': mse_decision_tree,
    'R-squared (R2)': r2_decision_tree,
    'Train RMSE': train_rmse_decision_tree,
    'Test RMSE': test_rmse_decision_tree,
    'Train MAE': train_mae_decision_tree,
    'Test MAE': test_mae_decision_tree,
    'Train R2': train_r2_decision_tree,
    'Test R2': test_r2_decision_tree
}, ignore_index=True)

<ipython-input-48-22b39d54e133>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [49]:
# Random Forest Regression Model
random_forest_model = RandomForestRegressor(random_state=42)
random_forest_model.fit(X_train_scaled, y_train)
y_pred_random_forest = random_forest_model.predict(X_test_scaled)
mse_random_forest = mean_squared_error(y_test, y_pred_random_forest)
r2_random_forest = r2_score(y_test, y_pred_random_forest)
train_rmse_random_forest = mean_squared_error(y_train, random_forest_model.predict(X_train_scaled), squared=False)
test_rmse_random_forest = mean_squared_error(y_test, y_pred_random_forest, squared=False)
train_mae_random_forest = mean_absolute_error(y_train, random_forest_model.predict(X_train_scaled))
test_mae_random_forest = mean_absolute_error(y_test, y_pred_random_forest)
train_r2_random_forest = r2_score(y_train, random_forest_model.predict(X_train_scaled))
test_r2_random_forest = r2_score(y_test, y_pred_random_forest)

print(test_r2_random_forest)

results_df = results_df.append({
    'Model': 'Random Forest Regression',
    'Mean Squared Error (MSE)': mse_random_forest,
    'R-squared (R2)': r2_random_forest,
    'Train RMSE': train_rmse_random_forest,
    'Test RMSE': test_rmse_random_forest,
    'Train MAE': train_mae_random_forest,
    'Test MAE': test_mae_random_forest,
    'Train R2': train_r2_random_forest,
    'Test R2': test_r2_random_forest
    }, ignore_index=True)

0.587632474577162


<ipython-input-49-c614408c14db>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [50]:
# Display the results DataFrame
results_df


,Model,Mean Squared Error (MSE),R-squared (R2),Train RMSE,Test RMSE,Train MAE,Test MAE,Train R2,Test R2
0,Linear Regression,384.446498,0.487827,19.967085,19.607307,12.988094,12.403136,0.580773,0.487827
1,SVM Regressor,479.099573,0.361727,26.275386,21.888343,15.474728,11.996981,0.274031,0.361727
2,Decision Tree Regression,550.145299,0.267077,0.000000,23.455176,0.000000,14.658120,1.000000,0.267077
3,Random Forest Regression,309.530493,0.587632,6.879408,17.593479,4.362073,11.078205,0.950235,0.587632
